In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
read_csv=pd.read_csv('C:/Users/dell/Desktop/DATA7901/Final_proposal/dataset_space_API_past_launches_raw.csv')
read_csv.dtypes

#Data wrangling
falcon9_data = read_csv[read_csv['BoosterVersion'] == 'Falcon 9']
mean_value=round(falcon9_data['PayloadMass'].mean(),2)
falcon9_data_new= falcon9_data.loc[:,'PayloadMass'].fillna(mean_value,inplace=True)
falcon9_data

# the number and occurrence of each orbit
df1=falcon9_data.groupby(['BoosterVersion','Orbit'])['Orbit'].count().sort_values(ascending=False)
df1

#the number and occurrence of mission outcome per orbit type
df2=falcon9_data.groupby(['Orbit','Outcome'])['Outcome'].count().reset_index(name='Count').sort_values(['Orbit','Count'], ascending=False)
print(df2)

#Classfiy successful landing and fail landing
falcon9_data['Outcome'].astype(str)
landing_status=[]
for row in falcon9_data['Outcome']:
        if row in ("True Ocean", "True RTLS","True ASDS"):landing_status.append("Success")
        elif row in ('False Ocean','False RTLS','False ASDS','None ASDS','None None'):landing_status.append("Fail")
        else: landing_status.append("No Rating")
falcon9_data['LandingStatus'] = landing_status
falcon9_data['LandingStatus'].astype(str)
falcon9_data

#Success and fail launch
df3=falcon9_data.groupby(['BoosterVersion','LandingStatus'])['LandingStatus'].count()
print(df3)
#The number of Each orbit's successful and fail landing 
df4=falcon9_data.groupby(['Orbit','LandingStatus'])['LandingStatus'].count().reset_index(name='Count').sort_values(['Orbit','Count'], ascending=False)
print(df4)

    Orbit      Outcome  Count
30   VLEO    True ASDS     12
29   VLEO   False ASDS      2
28    SSO    True RTLS      3
27    SSO    True ASDS      2
26     SO    None None      1
24     PO    True ASDS      5
21     PO   False ASDS      1
22     PO  False Ocean      1
23     PO    None None      1
25     PO   True Ocean      1
20    MEO    True ASDS      2
19    MEO    None None      1
18    LEO    True RTLS      4
16    LEO    None None      2
17    LEO   True Ocean      1
15    ISS    True RTLS      7
13    ISS    True ASDS      5
12    ISS    None None      3
8     ISS   False ASDS      2
9     ISS  False Ocean      1
10    ISS   False RTLS      1
11    ISS    None ASDS      1
14    ISS   True Ocean      1
7     HEO    True ASDS      1
5     GTO    True ASDS     13
4     GTO    None None     11
2     GTO   False ASDS      1
3     GTO    None ASDS      1
6     GTO   True Ocean      1
1     GEO    True ASDS      1
0   ES-L1   True Ocean      1
BoosterVersion  LandingStatus
Falcon 9  

D:\Program\Anaconda\lib\site-packages\pandas\core\generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
C:\Users\dell\AppData\Local\Temp/ipykernel_10532/1880435828.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  falcon9_data['LandingStatus'] = landing_status


In [2]:
#Interactive map
location_data=falcon9_data[['FlightNumber','LaunchSite','LandingStatus','Longitude','Latitude']]
location_map=folium.Map(location=[28.561857,-80.577366], zoom_start=20, control_scale=True)
def status_color(status):
    if status == 'Success':
        return 'blue'
    else:
        return 'red'

marker_cluster = MarkerCluster(name="launch status")
location_map.add_child(marker_cluster)
location_data['marker_color'] = location_data['LandingStatus'].apply(status_color)
#mCluster_success = MarkerCluster(name="success launch").add_to(location_map)
#mCluster_fail = MarkerCluster(name="fail launch").add_to(location_map)

for index, row in location_data.iterrows():
    # create and add a Marker cluster to the site map
    location = [row['Latitude'], row['Longitude']]
    html =   "Name:"+row['LaunchSite']+"<br>Status:"+row['LandingStatus']
    iframe = folium.IFrame(html, width=300, height=130)
    popup = folium.Popup(iframe, max_width=300)
    folium.map.Marker(location, popup=popup,icon=folium.Icon(color='white',icon_color=row['marker_color'])).add_to(marker_cluster)

for index, location_info in location_data.iterrows():
    folium.Marker(location=[location_info['Latitude'], location_info['Longitude']], popup=location_info["LaunchSite"]).add_to(location_map)
    folium.Circle(radius=1000,location=[location_info['Latitude'], location_info['Longitude']],popup=location_info["LaunchSite"], 
                  ccolor='#3186cc',opacity=0.8,fill=False).add_to(location_map)
folium.LayerControl().add_to(location_map)   
"""for row in location_data.itertuples():
    marker = folium.Marker(location=[row.Latitude, row.Longitude], popup=popup,fill_color='yellow', axis=1)
    if row.LandingStatus == "Success":
        mCluster_success.add_child(marker)
    elif row.LandingStatus == "Fail":
        mCluster_fail.add_child(marker)        """
location_map

C:\Users\dell\AppData\Local\Temp/ipykernel_10532/776420179.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  location_data['marker_color'] = location_data['LandingStatus'].apply(status_color)
